In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

### 1. 문서 로드

In [2]:
file_path = "../00_data/Sustainability_report_2024_kr.pdf"

print("문서로드중")
loader = PyPDFLoader(file_path)
docs = loader.load()

문서로드중


In [3]:
len(docs)

83

In [4]:
docs[:10][1]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 1, 'page_label': '2'}, page_content='A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024\nCEO 메시지\n회사 소개\n이해관계자 소통\nOur Company\n04\n05\n06\n준법과 윤리경영\nPrinciple\n53\n중대성 평가\nMateriality Assessment\n08\n임직원\n공급망\n사회공헌\n개인정보보호/보안\n고객의 안전/품질\nPeople\n31\n39\n45\n48\n50\n경제성과\n사회성과\n환경성과\n지역별 수자원 현황   \n사업부문별 환경성과\nFacts & Figures\n56\n57\n62\n65\n66\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서\nScope 3 온실가스 배출량 검증 의견서\nGRI Index\nTCFD 대조표\nSASB 대조표\n전사차원의 기후변화 대응 협력 활동\nAbout This Report\nAppendix\n70\n71\n72\n74\n77\n79\n81\n82\n[DX부문] \n추진체계 및 주요성과\n기후변화\n자원순환\n수자원 및 오염물질\n[DS부문]  \n추진체계 및 주요성과 \n기후변화\n수자원\n폐기물\n오염물질\nPlanet\n12\n13\n15\n17\n19\n20\n23\n26\n28\n삼성전자 지속가능경영보고서 2024 02Our C

### 2. 텍스트 스플리터(청킹 = 조각내다)

In [5]:
# 1. RecursiveCharacterTextSplitter 객체 생성
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,     # 한 조각(chunk)의 최대 문자 수 (대략 1000자 단위로 자름)
    chunk_overlap = 100,   # 조각 간에 겹치는 문자 수 (문맥 유지를 위해 앞뒤로 100자 겹치게 함)
    separators = ["\n\n", "\n", " ", ""]  # 텍스트를 자를 때 우선적으로 고려할 구분자들 (단락 → 줄바꿈 → 공백 → 글자 단위)
)

# 2. 문서(docs)를 여러 개의 chunk로 분리
chunks = splitter.split_documents(docs)

# 3. 전체 잘린 조각(chunk)의 개수 출력
print(len(chunks), "전체 잘린 chunk 사이즈")

207 전체 잘린 chunk 사이즈


In [6]:
print(chunks[10].page_content)

· 업종 간 협력
·  기후 대응을 포함한  
UN SDGs에 대한 기여
·  투명하고 신속한 정보 공개
·  기업 간담회
·  NGO 미팅
·  이해관계자 포럼
·  시민사회 - 경영진 간담회
·  노동인권 이해관계자 워크샵
·  지속가능경영 웹사이트
· 산업협회
·  글로벌 NGO 대상 의견 수렴
·  RBA
1)
, RMI
2)
, BSR
3)
 활동
1) Responsible Business Alliance
2) Responsible Minerals Initiative
3) Business for Social Responsibility
·  UNGC
1)
·  ACEC
2)
, SCC
3)
 활동
1) United Nations Global Compact
2) Asia Clean Energy Coalition
3) Semiconductor Climate Consortium
정부 ·  간접 경제효과(투자, 고용 등 파생효과)
·  공정거래
·  안전·보건
·  컴플라이언스
·  기업윤리
·  정책 간담회
·  국회
·  정책수립 공청회
·  정책자문기구
·  지속가능경영 웹사이트
·  정부와 협업하여 중소기업 지원 프로그램 운영 ·  정부와 협업하여 벤처투자 창구 설립·운영
언론 ·  주요 제품/사업 실적 및 전략
·  투자, R&D, M&A,  
신사업 등 미래 성장 전략 
·  탄소중립 등 ESG 추진 성과
·  인/노사, 환경안전, 특허,  
제품·서비스 품질 등
·  보도자료
·  지속가능경영 웹사이트
·  삼성전자 반도체 뉴스룸
·  삼성전자 뉴스룸
· 미디어 간담회
·  글로벌 IT 전시회·신제품 발표 취재지원
·  미디어데이 개최
·  기자회견
·  인터뷰
·  기획홍보
 Our Company


### 3. 임베딩 생성 chromadb 저장

In [ ]:
db_path = "../07_vectorstore/chromadb_rag_basic"   # 벡터 저장소를 저장할 경로 지정

embedding = OpenAIEmbeddings(model="text-embedding-3-small")   # 텍스트를 벡터로 변환할 임베딩 모델 설정

vectorstore = Chroma.from_documents(   # 문서 조각(chunks)을 벡터화하여 저장소 생성
    documents=chunks,                  # 분할된 문서 조각 리스트
    embedding=embedding,               # 사용할 임베딩 모델
    persist_directory=db_path,         # 벡터 데이터를 저장할 폴더 경로
    collection_name="samsung_2024"     # 데이터 컬렉션 이름 지정
)

print("벡터 저장소 저장 완료")   # 벡터 저장소 생성이 완료되었음을 출력

벡터 저장소 저장 완료


### 4. 검색기 구성(retriever)

In [8]:
retriever = vectorstore.as_retriever()   # 벡터 저장소를 검색기(retriever)로 변환
result = retriever.invoke("이 보고서에서 2024년 주요 지속 가능 경영 목표가 무엇인가요?")   # 질문과 가장 관련 있는 문서 조각 검색
result   # 검색된 문서 조각(결과) 출력

[Document(metadata={'page_label': '4', 'trapped': '/False', 'page': 3, 'producer': 'Adobe PDF Library 15.0', 'source': '../00_data/Sustainability_report_2024_kr.pdf', 'creationdate': '2024-11-25T11:10:32+09:00', 'total_pages': 83, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'moddate': '2024-11-25T11:10:46+09:00'}, page_content='삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가

In [9]:
for item in result:    # 검색 결과(result)에 있는 각 문서 조각을 하나씩 반복
    print(item.page_content[:50])  # 각 조각의 텍스트 앞 50글자만 출력
    print("-"*50)      # 구분선을 출력해 각 조각을 시각적으로 구분


삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateria
--------------------------------------------------
삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateria
--------------------------------------------------
삼성전자 지속가능경영보고서 2024
04
Our Company AppendixMateria
--------------------------------------------------
삼성전자 지속가능경영보고서 2024
60
Our Company AppendixMateria
--------------------------------------------------


### 5. 기본 체인 만들기

In [34]:
# 1. 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     주어진 컨텍스트만 근거로 간결하게 정확하게 대답해라
     컨텍스트에 없으면 문서에 "근거 없음"이라고 말해라
     
     [컨텍스트]
     {context}
     """),
     ("human", "{style}대로 {question}에 대해 대답해")
])

# 모델 선택
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature= 0
)

# 3. outputparser 선택
outputparser = StrOutputParser()

# 4. chain 생성
chain = rag_prompt | model | outputparser

In [33]:
# 테스트용
result[2].page_content

'삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능성 보고지침\n(CSRD)과 미국 증권거래위원회(SEC) 기후공시 기준 역시 세부 내용을 \n순차적으로 확정하며 ESG 정보의 의무 공시 시대가 열리고 있습니다.\n이와 함께, EU 탄소국경조정제도(CBAM)와 EU 배터리규제 등을 통한 \n환경규제 역시 지속 강화되는 추세이고, 독일에서는 공급망의 인권과 \n근로환경 관리를 의무화하는 공급망실사법이 2023년 발효되었으며, \n2024년 5월 EU 공급망 실사지침(CSDDD)이 확정되는 등 인권 분야에 \n대한 관심 또한 지속 고조되고 있습니다. \n삼성전자는 이러한 추세에 맞춰 지속가능한 미래를 위한 노력을 계속해'

### 6. RAG 체인 만들기

In [20]:
# 문서 조각들을 하나의 문자열로 합치는 함수
def format_docs(docs):
    tmp_docs = []                           # 각 문서 조각의 텍스트를 임시로 저장할 리스트
    for item in docs:                       # docs 리스트의 각 문서 조각 반복
        tmp_docs.append(item.page_content)  # 각 조각의 텍스트(page_content)만 추출해 리스트에 추가
    return "\n\n---\n\n".join(tmp_docs)     # 문서 조각들을 구분선("\n\n---\n\n")으로 연결하여 하나의 문자열로 반환

In [36]:
# rag_chain 만들기
rag_chain = (
    {"context" : RunnableLambda(lambda x : x["question"]) | retriever | RunnableLambda(format_docs),
    "style" : RunnableLambda(lambda x : x["style"]),
    "question" : RunnableLambda(lambda x : x["question"])
    }
    | chain
)
rag_chain

{
  context: RunnableLambda(...)
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000236B6BF6410>, search_kwargs={})
           | RunnableLambda(format_docs),
  style: RunnableLambda(...),
  question: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['context', 'question', 'style'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n     주어진 컨텍스트만 근거로 간결하게 정확하게 대답해라\n     컨텍스트에 없으면 문서에 "근거 없음"이라고 말해라\n\n     [컨텍스트]\n     {context}\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'style'], input_types={}, partial_variables={}, template='{style}대로 {question}에 대해 대답해'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000236BE7268D0>

In [40]:
result = rag_chain.invoke(
    {"style" : "냥냥체", 
     "question" :"중대성 평가의 정의는?"
    }
)
print(result)

냥냥~ 중대성 평가의 정의는, 삼성전자가 지속가능경영에 중요하게 고려해야 할 주제를 정할 때, 회사 활동이 외부 환경에 미치는 영향이랑 그 주제와 관련된 외부 요인이 회사에 재무적으로 미치는 영향 둘 다 살펴보는 거냥!
